In [1]:
from langchain.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.output_parsers import ResponseSchema, StructuredOutputParser


In [2]:
llm = Ollama(
    model="agent_pm", callback_manager=CallbackManager([StreamingStdOutCallbackHandler()])
)

llm_json = Ollama(
    model="agent_json_fixer", callback_manager=CallbackManager([StreamingStdOutCallbackHandler()])
)

In [7]:
tasks = ResponseSchema(
        name="tasks",
        description="The Project split into tasks, as a bulletpoint list.",
    )
timeline = ResponseSchema(
        name="timeline",
        description="The time required for each task, as a bulletpoint list.",
    )
deliverables =  ResponseSchema(
        name="deliverables",
        description="The Project deliverables with scope, as a bulletpoint list.",
    )
team = ResponseSchema(
        name="team",
        description="The required Project team's skills and experience level, as a bulletpoint list.",
    )
risks = ResponseSchema(
        name="risks",
        description="The Project risks, as a bulletpoint list.",
    )
budget = ResponseSchema(
        name="budget",
        description="The budget for each task, as a bulletpoint list.",
    )
metrics = ResponseSchema(
        name="metrics",
        description="The Project metrics, as a bulletpoint list.",
    )
output_parser = StructuredOutputParser.from_response_schemas([
    tasks,
    timeline,
    deliverables,
    team,
    risks,
    budget,
    metrics,
])

response_format = output_parser.get_format_instructions()
print(response_format)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"tasks": string  // The Project split into tasks, as a bulletpoint list.
	"timeline": string  // The time required for each task, as a bulletpoint list.
	"deliverables": string  // The Project deliverables with scope, as a bulletpoint list.
	"team": string  // The required Project team's skills and experience level, as a bulletpoint list.
	"risks": string  // The Project risks, as a bulletpoint list.
	"budget": string  // The budget for each task, as a bulletpoint list.
	"metrics": string  // The Project metrics, as a bulletpoint list.
}
```


In [8]:
template = """\
You have been given the following project brief. Identify and plan the key project tasks step by step.

{format_instructions}

Project Brief:
{brief}
"""

brief = """\
- The project is for a big multinational firm.
- The firm employs about 10000 people.
- The firm is grouped into a hierarchical structure, where each group specialises in a different industry or competency.
- The firm is struggling to forecast its revenue accurately.
- We are proposing to use machine learning an algorithms coupled with sourcing external market data to improve forecasting.
- We aim to achieve a better accuracy than their current forecasting process.
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["brief"],
    partial_variables={"format_instructions": response_format}
)

_input = prompt.format_prompt(brief=brief)
output_plan = llm(_input.to_string())

```json
{
  "tasks": [
    "Research and identify relevant external market data sources",
    "Develop machine learning algorithms to integrate with firm's internal data",
    "Train and validate the model using historical data",
    "Implement the model into the firm's forecasting process",
    "Monitor and evaluate the performance of the new forecasting system",
    "Provide training and support to the firm's employees on the new system"
  ],
  "timeline": [
    "Research and identify relevant external market data sources: 2 weeks",
    "Develop machine learning algorithms to integrate with firm's internal data: 4 weeks",
    "Train and validate the model using historical data: 6 weeks",
    "Implement the model into the firm's forecasting process: 3 weeks",
    "Monitor and evaluate the performance of the new forecasting system: ongoing",
    "Provide training and support to the firm's employees on the new system: 2 weeks"
  ],
  "deliverables": [
    "List of external market data s

In [13]:
def parse_json_via_llm(string_to_parse):
    try:
        json_string = output_parser.parse(string_to_parse)
    except Exception as e:
        print("parsing failed, falling back to json formatter")
        json_template = """\
        You have been given the following text. Convert it to correct JSON. If text is duplicated keep the most detailed version. No other text or information should be provided.

        {format_instructions}

        Text to convert to JSON:
        {text_not_yet_json}
        """

        prompt = PromptTemplate(
            template=json_template,
            input_variables=["text_not_yet_json"],
            partial_variables={"format_instructions": response_format}
        )

        _input = prompt.format_prompt(text_not_yet_json=string_to_parse)
        json_output = llm_json(_input.to_string())
        json_string = output_parser.parse(json_output)
    return json_string

json_string = parse_json_via_llm(output_plan)
print(json_string)

{'tasks': ['Research and identify relevant external market data sources', "Develop machine learning algorithms to integrate with firm's internal data", 'Train and validate the model using historical data', "Implement the model into the firm's forecasting process", 'Monitor and evaluate the performance of the new forecasting system', "Provide training and support to the firm's employees on the new system"], 'timeline': ['Research and identify relevant external market data sources: 2 weeks', "Develop machine learning algorithms to integrate with firm's internal data: 4 weeks", 'Train and validate the model using historical data: 6 weeks', "Implement the model into the firm's forecasting process: 3 weeks", 'Monitor and evaluate the performance of the new forecasting system: ongoing', "Provide training and support to the firm's employees on the new system: 2 weeks"], 'deliverables': ['List of external market data sources identified', 'Machine learning algorithms developed and tested', 'Tra

In [11]:
template = """\
Given the following project brief, and a list of tasks to solve for the brief. Take each task in the list and plan it step by step in detail.

{format_instructions}

Project Brief:
{brief}

Tasks:
{tasks}
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["brief"],
    partial_variables={"format_instructions": response_format}
)

_input = prompt.format_prompt(brief=brief, tasks=json_string['tasks'])
task_plan = llm(_input.to_string())

Here are the detailed step-by-step plans for each task in the project brief:

Task 1: Research and identify relevant external market data sources

* Step 1: Identify the industries or competencies that the firm specialises in.
	+ Step 1.1: Create a list of potential external market data sources for each industry or competency.
	+ Step 1.2: Evaluate the relevance and accuracy of each source based on factors such as data quality, coverage, and frequency.
* Step 2: Shortlist the most relevant external market data sources for further analysis.
	+ Step 2.1: Analyze the shortlisted sources to determine their suitability for the project.
	+ Step 2.2: Select the best-suited sources based on factors such as data quality, coverage, and frequency.
* Step 3: Develop a plan for collecting and integrating the selected external market data sources with the firm's internal data.
	+ Step 3.1: Determine the format and structure of the external data sources.
	+ Step 3.2: Develop a plan for data integrati

In [14]:
json_task_string = parse_json_via_llm(task_plan)
print(json_task_string)

parsing failed, falling back to json formatter
```json
{
    "tasks": [
        {
            "name": "Research and identify relevant external market data sources",
            "steps": [
                {
                    "step": "Step 1",
                    "subSteps": [
                        {
                            "name": "Identify the industries or competencies that the firm specialises in",
                            "description": "Create a list of potential external market data sources for each industry or competency."
                        },
                        {
                            "name": "Evaluate the relevance and accuracy of each source based on factors such as data quality, coverage, and frequency.",
                            "description": "Shortlist the most relevant external market data sources for further analysis."
                        }
                    ]
                },
                {
                    "step": "Step 2",


KeyboardInterrupt: 